[View in Colaboratory](https://colab.research.google.com/github/rkhaled98/Karmalutional-Network/blob/master/scripts/KARMALUTIONALNETWORK.ipynb)

In [9]:
from google.colab import files
uploaded = files.upload()
# you have to download this file and upload it (unzip too): https://www.kaggle.com/devjyotichandra/glove6b50dtxt
# make sure to change the name to glove.6B.50d.txt

Saving glove.6B.50d.txt to glove.6B.50d.txt


In [10]:
from google.colab import files
uploaded = files.upload()
# get these files exactly from git repo under data folder: data/news_news_small_1.csv and data/news_news_small_test_1.csv


Saving news_news_small_1.csv to news_news_small_1.csv
Saving news_news_small_test_1.csv to news_news_small_test_1.csv


In [46]:
!ls # make sure there are the files required above in the folder

datalab  glove.6B.50d.txt  news_news_small_1.csv  news_news_small_test_1.csv


In [0]:
import csv
import numpy as np

def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map
  
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()
  
def read_csv(filename = 'data/emojify_data.csv'):
    phrase = []
    emoji = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[0])
            # print(phrase)
            emoji.append(row[1])

    X = np.asarray(phrase)
    # print(X)
    Y = np.asarray(emoji)

    return X, Y
  
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [0]:
def print_predictions(X, pred):
    print()
    for i in range(X.shape[0]):
        print(X[i], label_to_emoji(int(pred[i])))

In [0]:
def predict(X, Y, W, b, word_to_vec_map, output_prediction=False):
    """
    Given X (sentences) and Y (relative score value), predict the Y and get the accuracy.
    
    Arguments:
    X -- input data containing sentences, numpy array of shape (m, None)
    Y -- labels, containing relative score index, numpy array of shape (m, 1)
    
    Returns:
    pred -- numpy array of shape (m, 1) with your predictions
    """
    m = X.shape[0]
    pred = np.zeros((m, 1))
    
    for j in range(m):                       # Loop over training examples
        
        # Split jth test example (sentence) into list of lower case words
        words = X[j].lower().split()
        
        # Average words' vectors
        avg = np.zeros((50,))
        for w in words:
            avg += word_to_vec_map[w]
        avg = avg/len(words)

        # Forward propagation
        Z = np.dot(W, avg) + b
        A = softmax(Z)
        pred[j] = np.argmax(A)
        
        if output_prediction:
           print("The model predicts that the comment %s will receive a relative score ranking of %s" % (X[j], pred[j]))
        
    print("Accuracy: "  + str(np.mean((pred[:] == Y.reshape(Y.shape[0],1)[:]))))
    
    return pred

In [0]:
def predict_one(X, W, b, word_to_vec_map):
    words = X.lower().split()
    avg = np.zeros((50,))
    for w in words:
        avg += word_to_vec_map[w]
    avg = avg/len(words)
    Z = np.dot(W, avg) + b
    A = softmax(Z)
    pred = np.argmax(A)
    print("The model predicts that the comment: \n'%s'\n will receive a relative score ranking of: %s" % (X, pred))

In [21]:
print("reading training data")
X_train, Y_train = read_csv('news_news_small_1.csv') # the data needed by the read_csv is a csv with comment,rank generated by the datasets main script 
print("done reading training data")
filtered = list(filter(None, Y_train))
Y_train = np.asarray(filtered)
Y_train = Y_train.astype(int)
#print(Y_train) make sure theyre ints
print("done reading testing data")
X_test, Y_test = read_csv('news_news_small_test_1.csv')
print("done reading testing data")
filtered = list(filter(None, Y_test))
Y_test = np.asarray(filtered)
Y_test = Y_test.astype(int)
#print(Y_test) make sure theyre ints

# Y_train = np.append(Y_train, [0])
print(X_train.shape, Y_train.shape)
# Y_test = np.append(Y_test, [0])
print(X_test.shape, Y_test.shape)

reading training data
done reading training data
done reading testing data
done reading testing data
(3852,) (3852,)
(442,) (442,)


In [0]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [23]:
print("reading glove vector")
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')
print("done reading glove vector")

reading glove vector
done reading glove vector


In [0]:
def remove_non_keys(X, dict):
    for i in range(0, len(X)):
        for word in X[i].split(" "):
            if (not (word in dict)):
                if (not (word == " ")):
                    if (not (word == "")):
                        X[i] = X[i].replace(word, '')
                        X[i] = " ".join(X[i].split())
    return X

In [25]:
print("removing non keys")
X_train = remove_non_keys(X_train, word_to_index)
X_train = remove_non_keys(X_train, word_to_index)
X_test = remove_non_keys(X_test, word_to_index)
X_test = remove_non_keys(X_test, word_to_index)
print("done removing non keys")

removing non keys
done removing non keys


In [0]:
def sentence_to_avg(sentence, word_to_vec_map):
  
    # MAKE SURE TO CHANGE THE DIMENSIONS IN HERE IF YOU CHANGE THE GLOVE VECTOR
    """
    Converts a sentence (string) into a list of words (strings). Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    Arguments:
    sentence -- string, one training example from X
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, numpy-array of shape (50,)
    """
    
    #Split sentence into list of lower case words (around 1 line)
    words = [i.lower() for i in sentence.split()]

    # Initialize the average word vector, should have the same shape as your word vectors.
    avg = np.zeros((50,))
    
    #average the word vectors. You can loop over the words in the list "words".
    for w in words:
        avg += word_to_vec_map[w]
    avg = avg / len(words)
    
    return avg

In [0]:
def model(X, Y, word_to_vec_map, learning_rate = 0.001, num_iterations = 400):
    """
    Model to train word vector representations in numpy.
    
    Arguments:
    X -- input data, numpy array of comment sentences as strings, of shape (m, 1)
    Y -- labels, numpy array of integers between 0 and 9, numpy-array of shape (m, 1)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    learning_rate -- learning_rate for the stochastic gradient descent algorithm
    num_iterations -- number of iterations
    
    Returns:
    pred -- vector of predictions, numpy-array of shape (m, 1)
    W -- weight matrix of the softmax layer, of shape (n_y, n_h)
    b -- bias of the softmax layer, of shape (n_y,)
    """
    
    np.random.seed(1)

    # Define number of training examples
    m = Y.shape[0]                          # number of training examples
    n_y = 10                                 # number of classes  
    n_h = 50                                # dimensions of the GloVe vectors 
    
    # Initialize parameters using Xavier initialization
    W = np.random.randn(n_y, n_h) / np.sqrt(n_h)
    b = np.zeros((n_y,))
    
    # Convert Y to Y_onehot with n_y classes
    Y_oh = convert_to_one_hot(Y, C = n_y) 
    
    # Optimization loop
    for t in range(num_iterations):                       # Loop over the number of iterations
        for i in range(m):                                # Loop over the training examples
            
            # Average the word vectors of the words from the i'th training example
            avg = sentence_to_avg(X[i], word_to_vec_map)

            # Forward propagate the avg through the softmax layer
            z = np.dot(W, avg) + b
            a = softmax(z)

            # Compute cost using the i'th training label's one hot representation and "A" (the output of the softmax)
            cost = -np.sum(np.multiply(Y_oh[i], np.log(a)))
            
            # Compute gradients 
            dz = a - Y_oh[i]
            dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db = dz

            # Update parameters with Stochastic Gradient Descent
            W = W - learning_rate * dW
            b = b - learning_rate * db
        
        if t % 100 == 0:
            print("Epoch: " + str(t) + " --- cost = " + str(cost))
            pred = predict(X, Y, W, b, word_to_vec_map)

    return pred, W, b

In [54]:
pred, W, b = model(X_train, Y_train, word_to_vec_map)
print(pred)

print("Training set:")
pred_train = predict(X_train, Y_train, W, b, word_to_vec_map)
print('Test set:')
pred_test = predict(X_test, Y_test, W, b, word_to_vec_map)

Epoch: 0 --- cost = 2.531106730427029
Accuracy: 0.25778816199376947
Epoch: 100 --- cost = 2.527602392786885
Accuracy: 0.3110072689511942
Epoch: 200 --- cost = 2.5098161246682036
Accuracy: 0.3245067497403946
Epoch: 300 --- cost = 2.4991273871194726
Accuracy: 0.3304776739356179
[[9.]
 [1.]
 [1.]
 ...
 [9.]
 [7.]
 [1.]]
Training set:
Accuracy: 0.3291796469366563
Test set:
Accuracy: 0.334841628959276


In [50]:
predict_one("Rafi is the best", W, b, word_to_vec_map)
predict_one("Nanak is the best", W, b, word_to_vec_map)
predict_one("Colby is the best", W, b, word_to_vec_map)
predict_one("Brian is the best", W, b, word_to_vec_map)
predict_one("Kevin is the best", W, b, word_to_vec_map)
predict_one("Victor is the best", W, b, word_to_vec_map)
# predict_one("Victor is the greatest of all time", W, b, word_to_vec_map)

The model predicts that the comment: 
'Rafi is the best'
 will receive a relative score ranking of: 0
The model predicts that the comment: 
'Nanak is the best'
 will receive a relative score ranking of: 0
The model predicts that the comment: 
'Colby is the best'
 will receive a relative score ranking of: 1
The model predicts that the comment: 
'Brian is the best'
 will receive a relative score ranking of: 8
The model predicts that the comment: 
'Kevin is the best'
 will receive a relative score ranking of: 8
The model predicts that the comment: 
'Victor is the best'
 will receive a relative score ranking of: 5
